In this notebook, we find the solution of the daily English Wordle (https://www.nytimes.com/games/wordle/index.html).

In every iteration they tell you the best word to use, and you have to input the results, the colors. (Green='G', Yellow='Y', Gray='B').

In [1]:
#IMPORTS

import pandas as pd
import numpy as np

In [2]:
#DATASET LOADING

df = pd.read_csv('Datasets/valid_solutions.csv') #Only the words that can be the final solution
df3 = pd.read_csv('Datasets/valid_guesses.csv') #All the possible words to use

In [3]:
#We create a dictionary "diccionari" with every letter and it's frequency from the list of valid_solutions

big_list = []

for word in df['word']:
    list_of_letters = list(word)
    for lett in list_of_letters:
        big_list.append(lett)

        frequency = {}

for item in big_list:
    if item in frequency:
      frequency[item] += 1
    else:
      frequency[item] = 1
print(frequency)

diccionari = {k: v for k, v in sorted(frequency.items(), key=lambda item: item[1], reverse=True)}
diccionari

{'a': 979, 'b': 281, 'c': 477, 'k': 210, 's': 669, 'e': 1233, 't': 729, 'y': 425, 'o': 754, 'h': 389, 'r': 899, 'i': 671, 'd': 393, 'l': 719, 'u': 467, 'v': 153, 'n': 575, 'g': 311, 'p': 367, 'm': 316, 'f': 230, 'x': 37, 'w': 195, 'z': 40, 'j': 27, 'q': 29}


{'e': 1233,
 'a': 979,
 'r': 899,
 'o': 754,
 't': 729,
 'l': 719,
 'i': 671,
 's': 669,
 'n': 575,
 'c': 477,
 'u': 467,
 'y': 425,
 'd': 393,
 'h': 389,
 'p': 367,
 'm': 316,
 'g': 311,
 'b': 281,
 'f': 230,
 'k': 210,
 'w': 195,
 'v': 153,
 'z': 40,
 'x': 37,
 'q': 29,
 'j': 27}

In [4]:
#Dictionary inicialization

def init_diccs(llista_lletres):
    #---We create the first dictionary "diccionari" with every letter and it's frequency from the list of valid_solutions
    big_list = []

    for word in llista_lletres:
        list_of_letters = list(word)
        for lett in list_of_letters:
            big_list.append(lett)

            frequency = {}

    for item in big_list:
        if item in frequency:
          frequency[item] += 1
        else:
          frequency[item] = 1
    diccionari = {k: v for k, v in sorted(frequency.items(), key=lambda item: item[1], reverse=True)}
    
    
    #---We create a nested dictionary "diccionaris" with the frequency of every letter in each of the 5 positions  
    list5 = [[],[],[],[],[]]
    diccionaris = []

    for word in llista_lletres:
        list_of_letters = list(word)
        for i in range(len(list_of_letters)):
            list5[i].append(list_of_letters[i])

    frequency = {}
    for big_list in list5:
        frequency = {}
        for item in big_list:
            if item in frequency:
              frequency[item] += 1
            else:
              frequency[item] = 1
        diccionaris.append({k: v for k, v in sorted(frequency.items(), key=lambda item: item[1], reverse=True)})
    return diccionari, diccionaris

diccionari, diccionaris = init_diccs(df['word'])

In [5]:
#Given the list of possible words, we rank them giving highest scores to the ones who have letters with the most frequency
#And then we catch the word with the highest rank
    #There're two rankings that we combine to make the final ranking for the word:
        #- amount: Given diccionaris
        #- amount2: Given diccionari (and we later divide by 5 so they have the same scale)
        
def ranking(diccionaris, diccionari, possibilities):
    results_dicc = {}
    for word in possibilities:
        amount = 0
        amount2 = 0
        for letter,i in zip(word,[0,1,2,3,4]):
            try:
                amount += diccionaris[i][letter]
                amount2 += diccionari[letter]
            except:
                amount += 0
                amount2 += 0
        results_dicc[word] = (amount + amount2/5) * len(set(word))
    return max(results_dicc, key=results_dicc.get)

In [6]:
#This is the function that has all the rules of which words have to be deleted from the possibles, after your 5-letter input.

def main_function(paraula, resultats, ws1):
    ws11 = ws1.copy()
    for ws in ws1:
        for pos in range(5):
            if resultats[pos] == 'Y':
                
                if paraula[pos] == ws[pos]:
                    ws11.remove(ws)
                    break
                
                new = list(ws)
                new.pop(pos)
                if paraula[pos] not in new:
                    ws11.remove(ws)
                    break
                    
                elif paraula.count(paraula[pos]) > 1:
                    indexos = [m[0] for m in [(m.start(0), m.end(0)) for m in re.finditer(paraula[pos], paraula)]]
                    count_vg = 0
                    for ins in indexos:
                        if resultats[ins] in ['G','Y']:
                            count_vg += 1
                    if ws.count(paraula[pos]) < count_vg:
                        ws11.remove(ws)
                        break
                            

            elif resultats[pos] == 'B':
                wss = ws
                if paraula[pos] == ws[pos]:
                    ws11.remove(ws)
                    break
                
                elif paraula.count(paraula[pos]) > 1:
                    indexos = [m[0] for m in [(m.start(0), m.end(0)) for m in re.finditer(paraula[pos], paraula)]]
                    boolea = False
                    for ins in indexos:
                        if resultats[ins] == 'G':
                            boolea = True
                            wss = list(wss)
                            wss.remove(paraula[pos])
                        if resultats[ins] == 'Y':
                            wss = list(wss)
                            wss.remove(paraula[pos])
                if paraula[pos] in wss:
                    ws11.remove(ws)
                    break
                

            elif resultats[pos] == 'G':
                if ws[pos] != paraula[pos]:
                    ws11.remove(ws)
                    break
    #print(ws11, len(ws11))
    return ws11

In [ ]:
#WORDLESOLVER!!
#In this cell, we are getting the best word in each iteration until it finds the solution

all_possib = np.concatenate((df3['word'].unique(),df['word'].unique()))
paraula = ranking(diccionaris, diccionari, all_possib) #SOARE (best_first_word)
ws_solucions = list(df['word'].unique())
ws_tots = list(np.concatenate((df3['word'].unique(),df['word'].unique())))

while len(ws_solucions) > 2:
    print("Insert the word", paraula.upper(), "and write in capital letter the colors obtained: (Green='G', Yellow='Y', Gray='B').")
    resultats = input()
    ws_tots = main_function(paraula, resultats, ws_tots)
    ws_solucions = main_function(paraula, resultats, ws_solucions)
    if len(ws_solucions) > 2:
        if len(ws_solucions) > 5:
            paraula = ranking(diccionaris, diccionari, ws_tots)
        elif len(ws_solucions) <= 5:
            paraula = ranking(diccionaris, diccionari, ws_solucions)
        
if len(ws_solucions) == 1:
    print("Congratulations, today's word is:", ws_solucions[0].upper())
elif len(ws_solucions) == 2:
    print("It has to be one of these 2 words, try them!", list(map(lambda x: x.upper(), ws_solucions)))
elif len(ws_solucions) == 0:
    print("You have made a mistake while typing the results, rerun the cell")